In [88]:
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder,StandardScaler,OrdinalEncoder
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV
import joblib
import shap

In [92]:
df = pd.read_csv('data.csv').drop(columns=['Unnamed: 32','id']).drop_duplicates()

In [93]:
df['diagnosis'].value_counts()

B    357
M    212
Name: diagnosis, dtype: int64

In [96]:
features = ['radius_mean', 'texture_mean', 'perimeter_mean',
    'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
    'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean',
    'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
    'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
    'fractal_dimension_se', 'radius_worst', 'texture_worst',
    'perimeter_worst', 'area_worst', 'smoothness_worst',
    'compactness_worst', 'concavity_worst', 'concave points_worst',
    'symmetry_worst', 'fractal_dimension_worst']

In [98]:
X = df.drop(columns='diagnosis')
y = df["diagnosis"]

In [99]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.3, random_state=42)

In [102]:
grid={"C":np.logspace(-3,3,7), "penalty":["l1","l2"], "max_iter":[100, 500, 1000]}
model=LogisticRegression()
search=GridSearchCV(model,grid,cv=10)

In [103]:
search.fit(X_train, y_train)

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the doc

GridSearchCV(cv=10, estimator=LogisticRegression(),
             param_grid={'C': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03]),
                         'max_iter': [100, 500, 1000],
                         'penalty': ['l1', 'l2']})

In [104]:
search.best_params_

{'C': 100.0, 'max_iter': 500, 'penalty': 'l2'}

In [105]:
search.best_score_

0.9595512820512819

In [80]:
lr = LogisticRegression(C= 100.0, penalty= 'l2', max_iter=500)

In [81]:
pipe = Pipeline([
        ('scaler',StandardScaler()),
        ('model', lr)  
    ])

In [106]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('model', LogisticRegression(C=100.0, max_iter=500))])

In [110]:
explainer = shap.explainers.Linear(lr, masker=X_train)

In [84]:
print("score",pipe.score(X_test,y_test))

score 0.9590643274853801


In [30]:
logreg=LogisticRegression(C=0.001,penalty="l2")
logreg.fit(X_train,y_train)
print("score",logreg.score(X_test,y_test))

score 0.9191919191919192


/Users/laetitiataddei/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [31]:
joblib.dump(logreg, "lr_model.pkl")

['lr_model.pkl']

In [87]:
joblib.dump(pipe, "lr_model.joblib")

['lr_model.joblib']

In [89]:
explainer = shap.Explainer(pipe)
shap_values = explainer(X_train)

TypeError: The passed model is not callable and cannot be analyzed directly with the given masker! Model: Pipeline(steps=[('scaler', StandardScaler()),
                ('model', LogisticRegression(C=100.0, max_iter=500))])